# Dataset: *sydneysiege*

- df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-df.csv',index=False)
- src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-src.csv',index=False)

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-df.csv', encoding = 'utf-8', header = 0)
df.dropna(inplace = True)
df.reset_index(drop=True,inplace=True)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-src.csv', encoding = 'utf-8', header =0)

In [20]:
src_tw_df[src_tw_df.cleaned_src_tw.isnull()].cleaned_src_tw

158    NaN
287    NaN
347    NaN
680    NaN
Name: cleaned_src_tw, dtype: object

## Emotion Analysis

### T5 Based
<https://huggingface.co/mrm8488/t5-base-finetuned-emotion?text=I+wish+you+were+here+but+it+is+impossible>

In [3]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [4]:
t5_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion",use_fast=False)
t5_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")



def get_emotion_t5(text):
    input_ids = t5_tokenizer.encode(text , return_tensors='pt')
    output = t5_model.generate(input_ids=input_ids,max_length=2)
  
    dec = [t5_tokenizer.decode(ids) for ids in output]
    label = dec[0]
    return label

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\transformers\models\auto\modeling_auto.py:694: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [8]:
emotion_t5 = df.cleaned_reply_tw.apply(get_emotion_t5)

In [9]:
df['reply_emo_t5'] = emotion_t5


In [35]:
df['reply_emo_t5'] = df['reply_emo_t5'].apply(lambda x : re.sub('<pad> ',"",x))
df.reply_emo_t5

0         fear
1          joy
2        anger
3        anger
4        anger
         ...  
21377     fear
21378    anger
21379    anger
21380      joy
21381      joy
Name: reply_emo_t5, Length: 21382, dtype: object

In [27]:
src_emo_t5 = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_t5)
src_tw_df['src_emo_t5'] = src_emo_t5

In [34]:
import re
src_tw_df['src_emo_t5']  = src_tw_df.src_emo_t5.apply(lambda x : re.sub('<pad> ',"",x))

In [ ]:
ax = df.reply_emotion.value_counts().plot.bar()

for i in ax.patches:
    height = i.get_height()
    ax.text(
        i.get_x() + i.get_width() / 2,height, str(height), ha="center", va="bottom"
    )
plt.xticks(rotation=0)
plt.show()
#plt.savefig('D:/論文/PHEME9/Code/graph/sydneysiege/rep-emotion.pdf',dpi=300)

### roBERTa-base model
<https://huggingface.co/cardiffnlp/twitter-roberta-base-emotion>

In [21]:
import numpy as np
from scipy.special import softmax

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
ro_tokenizer = AutoTokenizer.from_pretrained(MODEL)
ro_model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [26]:
def get_emotion_roBERT(text):
    
    def preprocess(txt):
        new_text = []
        for t in txt.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)
    
    labels = ['anger','joy','optimism','sadness']
    text = preprocess(text)
    encoded_input = ro_tokenizer(text, return_tensors='pt')
    output = ro_model(**encoded_input)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    emo = labels[ranking[0]]
    return emo

In [28]:
emotion_ro = df.cleaned_reply_tw.apply(get_emotion_roBERT)

In [37]:
df['reply_emo_ro'] = emotion_ro
df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-df-2.csv',index=False)

In [31]:
src_emo_ro = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_roBERT)
src_tw_df['src_emo_ro'] = src_emo_ro

In [36]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-src-2.csv',index=False)

 - robert : 'anger','joy','optimism','sadness'
 - t5: 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise' 
 - text2emotion: 'angry, 'fear', 'happy, 'sad', 'surprise'

In [46]:
print("original reply tw: ", df.reply_tweet[8])
print("cleaned reply tw: ", df.cleaned_reply_tw[8])

print('roBERT based: ',df.reply_emo_ro[8])
print('t5 based: ',df.reply_emo_t5[8])
print('text2emotion: ', df.reply_emo_te[8])

original reply tw:  @abcnewsSydney our rules of laws &amp; democracy were exploited by the islamic extremists
cleaned reply tw:  rule law amp democracy exploited islamic extremist
roBERT based:  anger
t5 based:  fear
text2emotion:  Surprise
